**We obtain our dataset by leveraging Reddit's PRAW API**

<!-- This is a better approach than using Push Shift because of this documented [error](https://github.com/iterative/aita_dataset/releases/tag/v.20.1.1). Push shift has a bug where recent posts are not properly queried. PRAW is slower but seems to be more accurate -->

Authentication information for Reddit developer account: 
developer account username: we_a_bot

developer account password: CS224Nredditscraper

App Name: 224n-aita 

App ID: M86H9TtDHZdTbo4IjWEUYg

Secret: x0JKk-Bx6qjY1sxEejGxfs9KW96ZNA




In [ ]:
!pip install praw
import praw
import requests
import json
import pandas as pd
import time
import csv
# https://asyncpraw.readthedocs.io/en/stable/getting_started/quick_start.html for async praw
# for normal praw: https://praw.readthedocs.io/en/latest/getting_started/quick_start.html#read-only

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.4/189.4 KB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 KB 6.6 MB/s eta 0:00:00


In [ ]:
client_id = "M86H9TtDHZdTbo4IjWEUYg"
client_secret = "x0JKk-Bx6qjY1sxEejGxfs9KW96ZNA"
username="we_a_bot"
password="CS224Nredditscraper"

reddit = praw.Reddit(
    client_id=client_id,
    client_secret=client_secret,
    password=password,
    user_agent="224n test script",
    username=username,
)



In [ ]:
print(reddit.read_only)
print(reddit.user.me())

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



False
we_a_bot


In [ ]:
subreddit = reddit.subreddit("AmItheAsshole")
print(subreddit.display_name)
print(subreddit.title)
# print(subreddit.description)

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



AmItheAsshole
Am I the Asshole? 


In [ ]:
# import logging

# handler = logging.StreamHandler()
# handler.setLevel(logging.DEBUG)
# for logger_name in ("praw", "prawcore"):
#     logger = logging.getLogger(logger_name)
#     logger.setLevel(logging.DEBUG)
#     logger.addHandler(handler)

In [ ]:
# The url for this seems slightly busted now. 
# this is the current api: https://pushshift.io/api-parameters/


# first_epoch= 1370000000
# last_epoch = 1660000000
# def getPushshiftData(after, before):
#     url = 'https://api.pushshift.io/reddit/submission/search/?sort_type=created_utc&sort=asc&subreddit=amitheasshole&after='+ str(after) +"&before"+str(before)
#     print(url)
#     r = requests.get(url)
#     data = json.loads(r.text)
#     print(data)
#     return data
    
# timestamps = list()
# post_ids = list()
# score = list()

# after = first_epoch
# while int(after) < last_epoch:
#     data = getPushshiftData(after,last_epoch)
#     print(data)
#     break
#     for post in data:
#         tmp_time = post['created_utc']
#         tmp_id = post['id']
#         timestamps.append(tmp_time)
#         post_ids.append(tmp_id)
#     after = timestamps[-1]
#     print([str(len(post_ids)) + " posts collected so far."])
#     time.sleep(0.1)

# # Write to a csv file
# d = {'id':post_ids, 'timestamp':timestamps}
# df = pd.DataFrame(d)
# df.to_csv("post_ids.csv", index=False)

https://api.pushshift.io/reddit/submission/search/?sort_type=created_utc&sort=asc&subreddit=amitheasshole&after=1370000000&before1660000000
{'detail': [{'loc': ['query', 'sort'], 'msg': "unexpected value; permitted: 'id', 'created_utc', 'score'", 'type': 'value_error.const', 'ctx': {'given': 'asc', 'permitted': ['id', 'created_utc', 'score']}}]}
{'detail': [{'loc': ['query', 'sort'], 'msg': "unexpected value; permitted: 'id', 'created_utc', 'score'", 'type': 'value_error.const', 'ctx': {'given': 'asc', 'permitted': ['id', 'created_utc', 'score']}}]}
{'detail': [{'loc': ['query', 'sort'], 'msg': "unexpected value; permitted: 'id', 'created_utc', 'score'", 'type': 'value_error.const', 'ctx': {'given': 'asc', 'permitted': ['id', 'created_utc', 'score']}}]}


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd drive/MyDrive/CS\ 224N/CS\ 224N\ Project
! ls # verify that you are in the right directory

In [ ]:
df = pd.read_csv("aita_clean.csv")
use_posts = df['id'].tolist()


In [ ]:
# print(use_posts)
top_comments = []
counter = 27485
f = open('comments.csv',"w")
writer = csv.writer(f, quoting=csv.QUOTE_ALL)
# 1/3 of doc: 32000
# 2/3 of doc: 60000
# 3/3 of doc: end

for idx in use_posts[counter:32000]:
  post = reddit.submission(idx)
  post.comments.replace_more(limit=4)
  flat_comments = post.comments.list()
  flat_comments.sort(key=lambda comment: comment.score, reverse=True)
  top_comment = flat_comments[0].body

  entry = [idx, top_comment]
  top_comments.append(entry)
  
  if counter % 1000 == 0:
    print(counter)
    d = {'id': [i[0] for i in top_comments], 'comments': [i[1] for i in top_comments] }
    df = pd.DataFrame(d)
    df.to_csv("/comments_batch_" +str(counter)+".csv", index=False) 
  counter += 1

print(top_comments)
df_comments = pd.DataFrame(top_comments, columns=['id','comments'])
df_comments.head()
f.close()

In [ ]:
# counter
# print(len(top_comments))
# print(top_comments)
# f = open('comments.csv',"w") 
# writer = csv.writer(f, quoting=csv.QUOTE_ALL)
# header = ["id","comments"]
# writer.writerow(header)
# for line in top_comments:
#   writer.writerow(line)


4209
[['1ytxov', 'There is nothing wrong with your original comment - you were kind of just rephrasing the post. You became an asshole because you got too defensive. The "my issue is more with the concept of infinity" comment wasn\'t attacking you, they were just clearing up what they meant. You then got snarky about it, which is why they got upset.  \nThen, your apologetic comment is the total flipside, which gives the impression that you weren\'t being sincere. However, there was no need for their reply to be so rude, (much worse than yours) even if you kind of started it. '], ['1yu29c', 'Yep.  You\'re the asshole.  If you want to make the commitment to work hard on improving your health and your diet, then that is your job.  If those who live with you want to support you by changing their lifestyle, too, that would be fabulous. But, it\'s not for you to dictate to them, or to throw away their property.   \n\nIf you felt that throwing the donuts away is going to help your parents, I 

In [ ]:
post.comments.list()[1].body

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



"NTA. Studying always comes before random kids that you're forced to hang around because of your family's mutual friendships."

In [ ]:
# f.close()
print(counter)
print(len(top_comments))
d = {'id': [i[0] for i in top_comments], 'comments': [i[1] for i in top_comments] }
df = pd.DataFrame(d)
df.to_csv("/content/drive/MyDrive/CS 224N/CS 224N Project/comments_batch_" +str(counter)+".csv", index=False) 

32000
4515
